In [53]:
# Import libraries
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
import numpy as np
import re
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import warnings
import random
warnings.filterwarnings("ignore")

In [69]:
# Import csv file
cities_path = "../data/hotels_items.csv"

In [70]:
# Create dataframe
cities_df = pd.read_csv(hotels_path, usecols = ["city"])

In [71]:
cities_df.head()

,city
0,"Mutterstadt, Germany"
1,"Passau, Germany"
2,"Hamburg, Germany"
3,"Stralsund, Germany"
4,"Wolfsburg, Germany"


In [72]:
# Sanity check
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 727491 entries, 0 to 727490
Data columns (total 1 columns):
city    727491 non-null object
dtypes: object(1)
memory usage: 5.6+ MB


In [73]:
cities_df.drop_duplicates(inplace = True)

In [74]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27194 entries, 0 to 727408
Data columns (total 1 columns):
city    27194 non-null object
dtypes: object(1)
memory usage: 424.9+ KB


In [75]:
city_list = cities_df["city"].tolist()

In [77]:
# # Create empty columns for the values that will be pulled through API
hotels_df = pd.DataFrame()
hotels_df["rating"] = np.nan
hotels_df["name"] = None
hotels_df["address"] = None
hotels_df["locality"] = None
hotels_df["landmarks"] = None
hotels_df["price"] = np.nan

In [76]:
# List lenght
len(city_list)

27194

In [81]:
for city in city_list:  
    # Launch url
    url = "https://www.hotels.com/"

    # Create a new Firefox session
    searchKey = city 
    checkInDate = '08/27/2020' #Format %m/%d/%Y
    checkOutDate = '08/29/2020' #Format %m/%d/%Y
    response = webdriver.Safari()
    response = implicitly_wait(10)
    response.get(url)
    searchKeyElement = response.find_elements_by_name("q-destination")
    checkInElement = response.find_elements_by_name("q-localised-check-in")
    checkOutElement = response.find_elements_by_id("q-localised-check-out")
    submitButton = response.find_elements_by_class_name("bcta cta-strong")
    if searchKeyElement and checkInElement and checkOutElement:
        searchKeyElement[0].clear()
        searchKeyElement[0].send_keys(searchKey)
        checkInElement[0].clear()
        checkInElement[0].send_keys(checkInDate)
        checkOutElement[0].clear()
        checkOutElement[0].send_keys(checkOutDate)
        submitButton[0].click()
        sleep(5)
        
    #Selenium hands the page source to Beautiful Soup
    soup = BeautifulSoup(response.page_source, "html.parser")
    hotels = soup.find("ol", class_ = "listings infinite-scroll-enabled")
    for index, hotel in enumerate(hotels[:150]): 
        time.sleep(random.randrange(0,5))  
        hotel_name = soup.find("a", class_ = "property-name-link").text.strip()
        hotel_rating = float(soup.find("div", class_ = "reviews-box resp-module").strong.text.split()[-1])
        hotel_address = soup.find("span", itemprop = "address").text 
        hotel_locality = soup.find("a", class_ = "map-link xs-welcome-rewards").text.strip()
        hotel_landmarks = soup.find("ul", class_ = "property-landmarks").text
        hotel_price = float(soup.find("a", class_ = "price-link").ins.text.strip())
        # Fill each column of the row with the values gotten from the API response
        hotels_df.at[index, "rating"] = hotel_rating
        hotels_df.at[index, "name"] = hotel_name
        hotels_df.at[index, "address"] = hotel_address
        hotels_df.at[index, "locality"] = hotel_locality
        hotels_df.at[index, "landmarks"] = hotel_landmarks
        hotels_df.at[index, "price"] = hotel_price
        print(city)
    response.quit()

SessionNotCreatedException: Message: Could not create a session: You must enable the 'Allow Remote Automation' option in Safari's Develop menu to control Safari via WebDriver.
